In [2]:
import numpy as np
import math
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
import json
#
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
from scipy.stats import truncnorm
#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#
import xgboost as xgb

2023-05-11 18:09:38.907724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
np.random.seed(42)
tf.random.set_seed
n_list = [10, 15]
epochs = {
    10: 1,
    15: 1
}

$$
\pi(x_n) = \sum_{i=1}^{n-1}\, [100(x_{i+1}-x_{i}^2)^2 + (1 - x_i)^2]
$$

In [4]:
def rosenbrock(x, n):
    sum = 0.0
    for i in range(n-1):
        a = 100 * ((x[i+1] -x[i]**2)**2)
        b = (1 - x[i])**2
        sum += a+b
    return sum

def get_truncated_normal(mean=5, sd=1, low=0, upp=10, size=None):
    """normal distribution between 0 and 10"""
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd).rvs(size)

def generate_x(n, k):
    """
    generate n dimensional normal distribution array in range [0, 10] (mit gaussian standard!!!)
    n: how many columns to generate
    k : how many rows to generate
    """
    x = np.array([get_truncated_normal(size=k) for _ in range(n)])
    return x

In [7]:
def create_data(n, k=50_000, verbose=True):
    X = generate_x(n=n, k=k)
    y = rosenbrock(X, n)
    columns_name = [f'x_{i}' for i in range(n)]
    #
    df = pd.DataFrame(X.T, columns=columns_name)
    df["y"] = y
    #
    if verbose:
        df.head()
        df.describe()
    #
    X = df.drop('y', axis = 1)
    y = df['y']
    return X, y

## Train neural network and XGBoost model

In [8]:
def build_and_compile_ann():
    model = keras.Sequential([
      layers.Dense(64, activation='tanh'), # tanh, sigmoid, R2 < 0, 
      layers.Dense(128, activation='tanh'),
      layers.Dense(1)
    ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
    return model

def train_ann(X_train, y_train, epochs):
    ann_model = build_and_compile_ann()
    history = ann_model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        verbose=0, epochs=epochs)
    return ann_model, history

def train_xgb(X_train, y_train):
    xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
    xgb_model.fit(X_train, y_train)
    return xgb_model

In [9]:
metadata = {}
for n in n_list:
    X, y = create_data(n)
    X_df_train, X_df_test, y_df_train, y_df_test = train_test_split(X, y, test_size=0.1)
    X_train = X_df_train.values
    y_train = y_df_train.values
    #
    print(f"Training models for {n} variables")
    xgb_model = train_xgb(X_train, y_train)
    ann_model, history = train_ann(X_train, y_train, epochs=epochs[n])
    #
    y_pred_ann = ann_model.predict(X_df_test)
    y_pred_xgb = xgb_model.predict(X_df_test.values)
    r2_ann = r2_score(y_df_test, y_pred_ann.flatten())
    r2_xgb = r2_score(y_df_test, y_pred_xgb)
    print(f"R2 score of ANN ({n} variables): ", r2_ann)
    print(f"R2 score of XGBoost ({n} variables): ", r2_xgb)
    #
    metadata_n = {'ANN_R2': r2_ann, 'XGB_R2': r2_xgb, 'epochs': epochs[n]}
    metadata[n] = metadata_n
    #
    ann_model.save(f'models/ANN_{n}')
    xgb_model.save_model(f'models/XGB_{n}.json')

with open('models/metadata.json', 'w') as file:
    json.dump(metadata, file)

print("Training complete")

Training models for 10 variables


2023-05-11 18:10:27.521186: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 18:10:27.524176: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


157/157 [==============================] - 3s 18ms/step
R2 score of ANN (10 variables):  -12.116012199700815
R2 score of XGBoost (10 variables):  0.972556327033647


INFO:tensorflow:Assets written to: models/ANN_10/assets


INFO:tensorflow:Assets written to: models/ANN_10/assets


Training models for 15 variables
157/157 [==============================] - 3s 18ms/step
R2 score of ANN (15 variables):  -18.073467568352996
R2 score of XGBoost (15 variables):  0.9553017201951887


INFO:tensorflow:Assets written to: models/ANN_15/assets


INFO:tensorflow:Assets written to: models/ANN_15/assets


Training complete


In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    #plt.ylim([0, 200])
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)

plot_loss(history)